In [ ]:
#pip install camelot-py
#pip install opencv-python
#pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install fpdf2

Note: you may need to restart the kernel to use updated packages.


In [102]:
import pandas as pd
import camelot
import numpy as np
import os
import glob
from fpdf import FPDF

In [3]:
def reading_pdf_from_folder(folder_name):
    exact_path=os.path.join(folder_name,"*.pdf")
    files=glob.glob(exact_path) 

    files.sort()


    if not files:
        print("no data in the folder")
        
    
    list_of_all_dfs=[]

    for file in files:
        try:
            tables=camelot.read_pdf(file,pages="all",flavor="lattice")
            temp_tables= [table.df for table in tables]

            if len(temp_tables)> 0:
                del temp_tables[0]
            else:
                print("There is only one table in this file")
            
            df_single_file= pd.concat(temp_tables,axis=0,ignore_index=True)
            list_of_all_dfs.append(df_single_file)
        except Exception as e:
            print(f"Erorr while proccesing file {file}, {e}")
    
    if len(list_of_all_dfs)>0:
        df_final=pd.concat(list_of_all_dfs,ignore_index=True,axis=0)
        print(f"Function sucessed, DF from {folder_name} has been created")
        return df_final
    else:
        print("No data to show")
        return pd.DataFrame()

Old_Data="Old Bank Data"

df_old_data=reading_pdf_from_folder(Old_Data)

New_data="New Bank Data"

df_new_data=reading_pdf_from_folder(New_data)











no data in the folder
No data to show
Function sucessed, DF from New Bank Data has been created


In [4]:

def first_data_cleaning(df):
    if len(df)>0:
        col_name=["Posting Date","Data of Transaction","Description","Amt","Acc Balance after opp"]
        df.columns=col_name
        df=df.replace("",np.nan)
        df.dropna(subset=["Amt"],inplace=True)
        df['Description'] = df['Description'].str.replace(r'\s+', ' ', regex=True).str.strip()
        df['Amt'] = df['Amt'].astype(str).str.replace(r'\s+', '', regex=True)
        df['Amt'] = df['Amt'].str.replace(',', '.')
        df['Amt'] = df['Amt'].astype(float)
        return pd.DataFrame(df)
    else:
        return pd.DataFrame(df)


def data_cleaning_for_gpt(df):
    """ This function is made for delating personal data from Bank Data to make it more safty while providing in LLM.
     This can be ommited if we know that we have secured interior Gen AI. This function is not dealting all peresonal staff.May be extended in the future
     as of now is better to dealaete Names,Surnames and adresess manually after exporting to xlsx """
    
    df["Description"]=df["Description"].str.replace(r"\d{26}","",regex=True)#bank acc
    df["Description"]=df["Description"].str.replace(r"\d{11}","",regex=True)# telephon no with direction 
    df["Description"]=df["Description"].str.replace(r"\d{9}","",regex=True)# telephon no without direction 
    return pd.DataFrame(df)


In [145]:
df_new_data=first_data_cleaning(df_new_data)
df_new_data=data_cleaning_for_gpt(df_new_data)

df_new_data.to_excel("Data_for_AI/data_to_update_mapping.xlsx",index=False)

#Take generted excel and go to specified Gemini/Chat GPT and use "Prompt for updating Dictionary every month" in prompts.txt
#This is for keep mapping up to date and minimize manual classifaction. 

In [6]:
df_old_data=first_data_cleaning(df_old_data)


In [7]:
df_complete=pd.concat((df_old_data,df_new_data),ignore_index=True)

df_complete.head(10)

,Posting Date,Data of Transaction,Description,Amt,Acc Balance after opp
0,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.97,"5 159,00"
1,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.21,"5 158,79"
2,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.11,"5 158,68"
3,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.20,"5 158,48"
4,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.30,"4 687,44"
5,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.17,"4 687,27"
6,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-1.51,"4 685,76"
7,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-1.76,"4 684,00"
8,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.11,"4 683,89"
9,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.68,"4 683,21"


In [124]:
# I am located in Poland so my data is in polish language 
#This filed should be replaced be updated mapping from AI tool whenevery we are adding new data to analysis

mapping_rules = {

    #Inter-acc movements
    'PRZELEW WEWNĘTRZNY PRZYCHODZĄCY GRZEGORZ STAŃCZYK':("Inter-acc movements","Internal Moves"), #tu mozna wstawic zmienna user name  zeby uogulic dla innych
    'REVOLUT':("Inter-acc movements","Internal Moves"),

    # Smart saver
    'PRZELEW NA TWOJE CELE': ('Smart saver', 'Smart saver'),

    # Income
    'WYNAGRODZENIE': ('Income', 'Salary'),
    "BLIK P2P-PRZYCHODZĄCY":('Income', 'BLIK'),
    'PRZELEW ZEWNĘTRZNY PRZYCHODZĄCY': ('Income', 'External Transfers'),
    'PRZELEW WEWNĘTRZNY PRZYCHODZĄCY': ('Income', 'External Transfers'),
    'WYPŁATA Z CELU':("Income","Transfer from Smart Saver"),
    
    # Eating out
    'MCDONALDS': ('Eating out', 'MCDONALDS'),
    'KFC': ('Eating out', 'KFC'),
    'KEBAB': ('Eating out', 'KEBAB'),
    'RESTAURANT': ('Eating out', 'Restauramt'),
    'KAWA': ('Eating out', "KAWA"),
    'AUTOMAT': ('Eating out', 'AUTOMAT'),
    'ASVENDING': ('Eating out', 'AUTOMAT'),
    'STOLOWKA': ('Eating out', 'Restauramt'),
    'BAR': ('Eating out', 'Restauramt'),
    'RESTAURACJA': ('Eating out', 'Restauramt'),
    'UBER EATS': ('Eating out', 'Delivery'),
    'PIZZA HUT': ('Eating out', 'Pizza'),
    'COSTA COFFEE': ('Eating out', 'KAWA'),

    # Sport
    'SIŁOWNIA': ('Sport', 'Gym'),
    'GYM': ('Sport', 'Gym'),
    'BASEN':("Sport","Basen"),

    # Daily Purchase
    'BIEDRONKA': ('Daily Purchase', 'Biednronka'),
    'LIDL': ('Daily Purchase', 'Lidl'),
    'ZABKA': ('Daily Purchase', 'Zabka'),
    'ŻABKA': ('Daily Purchase', 'Zabka'),
    'PIEKARNIA': ('Daily Purchase', 'Piekarnia'),
    'HERT': ('Daily Purchase', 'Piekarnia'),
    'SPOŻYWCZY': ('Daily Purchase', 'Other Stores'),
    'ALDI':('Daily Purchase',"Aldi"),
    'JULA':('Daily Purchase',"Other Stores"),
    'CARREFOUR':('Daily Purchase',"CARREFOUR"),
    'MROWKA':('Daily Purchase',"Mrowka"),
    'BIUROMIX':("Daily Purchase","Artykuly Biurowe"),
    'KSIEGARNIA':("Daily Purchase","Artykuly Biurowe"),
    'ROSSMANN': ('Daily Purchase', 'Rossmann'),
    'HEBE': ('Daily Purchase', 'Hebe'),
    'APTEKA': ('Daily Purchase', 'Pharmacy'),
    'EMPIK': ('Daily Purchase', 'Empik'),
    'ALLEGRO': ('Daily Purchase', 'Allegro'),

    # Subcription
    'NETFLIX': ('Subcription', 'NETFLIX'),
    'DISNEY': ('Subcription', 'DISNEY'),
    'GOOGLE': ('Subcription', 'GOOGLE'),
    'APPLE.COM': ('Subcription', 'APPLE'),
    'MICROSOFT': ('Subcription', 'MICROSOFT'),
    'SPOTIFY': ('Subcription', 'SPOTIFY'),
    'DISNAY+': ('Subcription', 'DISNEY'),
    'SPOTI': ('Subcription', 'SPOTIFY'),

    # Cloaths
    'SINSAY': ('Cloaths', 'Casual'),
    'LPP': ('Cloaths', 'Casual'),
    'PEPCO': ('Cloaths', 'Casual'),
    'ZARA': ('Cloaths', 'Casual'),
    'DECATHLON': ('Cloaths', 'Outdor'),
    'H&M': ('Cloaths', 'Casual'),

    # Technology
    'MEDIA MARKT': ('Technology', 'MEDIA MARKT'),
    'MEDIAEXPERT': ('Technology', 'MEDIAEXPERT'),
    'MEDIA EXPERT': ('Technology', 'MEDIAEXPERT'),
    'RTV': ('Technology', 'RTV'),

    # Fixed Costs
    'ORANGE': ('Fixed Costs', 'Telephone'),
    'CZYNSZ': ('Fixed Costs', 'Rent'),
    'RATY KREDYTU': ('Fixed Costs', 'Debt pay'),
    'RATA KREDYTU': ('Fixed Costs', 'Debt pay'),
    'MIESZKANIE': ('Fixed Costs', 'Rent'),
    'NJU':('Fixed Costs', 'Internet'),

    # Transport
    'PKP': ('Transport', 'Trains and Buses'),
    'KOLEJE': ('Transport', 'Trains and Buses'),
    'URBANCARD': ('Transport', 'Trains and Buses'),
    'BILETY': ('Transport', 'Trains and Buses'),
    'ORLEN': ('Transport', 'Fuel'),
    'SHELL': ('Transport', 'Fuel'),
    'BP-':('Transport', 'Fuel'),
    'STACJA': ('Transport', 'Fuel'),
    'JAKDOJADE.PL':('Transport', 'MPK'),
    'HYPEROIL':('Transport', 'Fuel'),
    'UBER': ('Transport', 'Taxi'),
    'BOLT': ('Transport', 'Taxi'),

    #Travels
    'AIRBNB':("Travels","Apartaments"),
    'BOOKING':("Travels","Apartaments"),
    "RYANAIR":("Travels","Fly"),

    #Other Outcomes:
    'BLIK P2P-WYCHODZĄCY':("Online outcomes","BLIK"),

    #Investing
    "XTB":("Investing","XTB"),
    
    #Entertaiment
    "EBILET":("Entertaiment","Events"),
    'CINEMA CITY': ('Entertaiment', 'Cinema'),
    'HELIOS': ('Entertaiment', 'Cinema'),

}

In [125]:
def adding_categories(description):
    
    desc_clean=" ".join(description.upper().split())


    for keyword,(main_cat,sub_cat) in mapping_rules.items():
        if keyword.upper() in desc_clean:
            final_main_cat=main_cat
            final_sub_cat=sub_cat
            break
        
        else:
            final_main_cat="Others"
            final_sub_cat="Unclasifided"
            

    return final_main_cat, final_sub_cat



    



    


In [126]:
df_complete[["Main_Category","Sub_Category"]]=df_complete.apply(lambda row:pd.Series(adding_categories(row["Description"])),
axis=1)

df_complete.head(10)

,Posting Date,Data of Transaction,Description,Amt,Acc Balance after opp,Main_Category,Sub_Category
0,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.97,"5 159,00",Smart saver,Smart saver
1,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.21,"5 158,79",Smart saver,Smart saver
2,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.11,"5 158,68",Smart saver,Smart saver
3,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.20,"5 158,48",Smart saver,Smart saver
4,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.30,"4 687,44",Smart saver,Smart saver
5,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.17,"4 687,27",Smart saver,Smart saver
6,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-1.51,"4 685,76",Smart saver,Smart saver
7,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-1.76,"4 684,00",Smart saver,Smart saver
8,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.11,"4 683,89",Smart saver,Smart saver
9,2025-12-01,2025-12-01,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.68,"4 683,21",Smart saver,Smart saver


In [ ]:
#df_complete is Data Frame created to manage and analysie my personal finance so as of now I will another data frame with
#random data. There AI genereted for this project purpouse. I have used promopt to update maping based on below fake data to test if 
#all is working as expected.Fake data is for 4 months just to make it eaiser later during visualizationa and caluclating stats

files=glob.glob(os.path.join("fake_data",'*.xlsx'))

list_of_fake_dfs=[]

for file in files:
    df=pd.read_excel(file)
    list_of_fake_dfs.append(df)
    
    
df_fake=pd.concat(list_of_fake_dfs,axis=0,ignore_index=True)

#use below to update mapping to add fake data
df_fake.to_csv("Data_for_AI/fake_combined_before_categorization.csv")








In [128]:
df_fake[["Main_Category","Sub_Category"]]=df_fake.apply(lambda row:pd.Series(adding_categories(row["Description"])),
axis=1)

df_fake.head(10)

,Posting Date,Data of Transaction,Description,Amt,Main_Category,Sub_Category
0,2026-04-02,2026-04-02,PRZELEW ZEWNĘTRZNY WYCHODZĄCY MIESZKANIE KWIECIEŃ,-3600.00,Fixed Costs,Rent
1,2026-04-02,2026-04-02,PRZELEW NA TWOJE CELE PRZELEW Z RACHUNKU MIESZ...,-36.00,Smart saver,Smart saver
2,2026-04-02,2026-04-02,ZAKUP PRZY UŻYCIU KARTY BIEDRONKA,-58.90,Daily Purchase,Biednronka
3,2026-04-03,2026-04-03,ZAKUP PRZY UŻYCIU KARTY ZABKA,-13.50,Daily Purchase,Zabka
4,2026-04-03,2026-04-03,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.13,Smart saver,Smart saver
5,2026-04-04,2026-04-04,ZAKUP PRZY UŻYCIU KARTY SHELL STACJA,-125.40,Transport,Fuel
6,2026-04-04,2026-04-04,ZAKUP PRZY UŻYCIU KARTY KFC,-30.50,Eating out,KFC
7,2026-04-05,2026-04-05,PRZELEW ZEWNĘTRZNY WYCHODZĄCY. SPLATA RATY KRE...,-110.65,Fixed Costs,Debt pay
8,2026-04-05,2026-04-05,PRZELEW NA TWOJE CELE SPLATA RATY KREDYTU,-1.11,Smart saver,Smart saver
9,2026-04-05,2026-04-05,ZAKUP PRZY UŻYCIU KARTY LIDL,-95.20,Daily Purchase,Lidl


In [147]:

df_fake.to_csv("Data_for_AI/fake_combined_after_categorization.csv")

In [148]:
df_complete.to_csv("Data_for_AI/df_complete_after_categorization.csv",index=False)

In [139]:
#statistical Analysis to get AI feedback and budget recomendation 

df_selected=df_fake
df_selected

,Posting Date,Data of Transaction,Description,Amt,Main_Category,Sub_Category
0,2026-04-02,2026-04-02,PRZELEW ZEWNĘTRZNY WYCHODZĄCY MIESZKANIE KWIECIEŃ,-3600.00,Fixed Costs,Rent
1,2026-04-02,2026-04-02,PRZELEW NA TWOJE CELE PRZELEW Z RACHUNKU MIESZ...,-36.00,Smart saver,Smart saver
2,2026-04-02,2026-04-02,ZAKUP PRZY UŻYCIU KARTY BIEDRONKA,-58.90,Daily Purchase,Biednronka
3,2026-04-03,2026-04-03,ZAKUP PRZY UŻYCIU KARTY ZABKA,-13.50,Daily Purchase,Zabka
4,2026-04-03,2026-04-03,PRZELEW NA TWOJE CELE ZAKUP PRZY UŻYCIU KARTY ...,-0.13,Smart saver,Smart saver
...,...,...,...,...,...,...
359,2026-01-15,2026-01-15,ZAKUP PRZY UŻYCIU KARTY LIDL,-58.70,Daily Purchase,Lidl
360,2026-01-19,2026-01-19,ZAKUP PRZY UŻYCIU KARTY SHELL,-135.20,Transport,Fuel
361,2026-01-23,2026-01-23,ZAKUP PRZY UŻYCIU KARTY PIZZA HUT,-85.00,Eating out,Pizza
362,2026-01-27,2026-01-27,ZAKUP PRZY UŻYCIU KARTY ZABKA,-17.60,Daily Purchase,Zabka


In [132]:

df_selected_income=df_selected[df_selected["Main_Category"]=="Income"]
df_selected_spedings=df_selected[df_selected["Main_Category"]!="Income"]
df_selected_spedings["Amt"]=df_selected_spedings["Amt"].abs()



/var/folders/lx/hzf7llr53d11btj66zf4sjnw0000gn/T/ipykernel_1395/4022134069.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_spedings["Amt"]=df_selected_spedings["Amt"].abs()


In [140]:
#Spendings in Main Category

df_Cat_spendings_to_analysis=df_selected_spedings.groupby("Main_Category").agg(
    Total_Spendings_in_Categoty=("Amt","sum"),
    Mean_Spendings_in_Categoty=("Amt","mean"),
    Min_value_in_Category=("Amt","min"),
    Max_value_in_Category=("Amt",'max')
).round(2)

#Spendings in Sub Category

df_SubCat_spendings_to_analysis=df_selected_spedings.groupby("Sub_Category").agg(
    Total_Spendings_in_SubCategoty=("Amt","sum"),
    Mean_Spendings_in_SubCategoty=("Amt","mean"),
    Min_value_in_SubCategory=("Amt","min"),
    Max_value_in_SubCategory=("Amt",'max'),
    Count_of_occurance=("Description","count")
).round(2)

#General Income accros Sub categories

df_SubCat_income_to_analysis=df_selected_income.groupby("Sub_Category").agg(
    Total_Income_in_SubCategoty=("Amt","sum"),
    Mean_Income_in_SubCategoty=("Amt","mean"),
    Min_value_in_SubCategory=("Amt","min"),
    Max_value_in_SubCategory=("Amt",'max'),
    Count_of_occurance=("Description","count")
).round(2)



In [141]:
df_SubCat_income_to_analysis

,Total_Income_in_SubCategoty,Mean_Income_in_SubCategoty,Min_value_in_SubCategory,Max_value_in_SubCategory,Count_of_occurance
Sub_Category,,,,,
BLIK,3538.00,221.12,14.00,1500.00,16
External Transfers,12376.00,2062.67,10.00,6000.00,6
Salary,4430.47,4430.47,4430.47,4430.47,1
Transfer from Smart Saver,451.00,150.33,8.00,387.00,3


In [149]:
#Preparing Pdf to analyze budget stats 
df_Cat_spendings_to_analysis=df_Cat_spendings_to_analysis.reset_index()
df_SubCat_spendings_to_analysis=df_SubCat_spendings_to_analysis.reset_index()
df_SubCat_income_to_analysis=df_SubCat_income_to_analysis.reset_index()

df_Cat_spendings_table_data = [df_Cat_spendings_to_analysis.columns.tolist()] +df_Cat_spendings_to_analysis.values.tolist()
df_SubCat_spendings_table_data = [df_SubCat_spendings_to_analysis.columns.tolist()] +df_SubCat_spendings_to_analysis.values.tolist()
df_SubCat_income_table_data=[df_SubCat_income_to_analysis.columns.tolist()] +df_SubCat_income_to_analysis.values.tolist()

pdf=FPDF()
pdf.add_page()
pdf.set_font("helvetica",size=13)
pdf.cell(txt="prompt",ln=True)
pdf.multi_cell(0,10,txt="prompt prompt",ln=True)

pdf.ln(20)

pdf.cell(txt="General Income accros Sub categories",ln=True)
pdf.ln(10)

with pdf.table(
    width=190,
    text_align="LEFT",
) as table:
    for data_row in df_SubCat_income_table_data:
        row=table.row()
        for obj in data_row:
            row.cell(str(obj))

pdf.ln(20)

pdf.cell(txt="Spendings in Main Categories",ln=True)
pdf.ln(10)

with pdf.table(
    width=190,
    text_align="LEFT",
) as table:
    for data_row in df_Cat_spendings_table_data:
        row=table.row()
        for obj in data_row:
            row.cell(str(obj))

pdf.ln(20)

pdf.cell(txt="Spendings in Sub Categories",ln=True)
pdf.ln(10)

with pdf.table(
    width=190,
    text_align="LEFT",
) as table:
    for data_row in df_SubCat_spendings_table_data:
        row=table.row()
        for obj in data_row:
            row.cell(str(obj))

pdf.output("Data_for_AI/Update promt for budget analyze.pdf")

/var/folders/lx/hzf7llr53d11btj66zf4sjnw0000gn/T/ipykernel_1395/1801948451.py:13: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(txt="prompt",ln=True)
/var/folders/lx/hzf7llr53d11btj66zf4sjnw0000gn/T/ipykernel_1395/1801948451.py:13: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(txt="prompt",ln=True)
/var/folders/lx/hzf7llr53d11btj66zf4sjnw0000gn/T/ipykernel_1395/1801948451.py:14: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(0,10,txt="prompt prompt",ln=True)
/var/folders/lx/hzf7llr53d11btj66zf4sjnw0000gn/T/ipykernel_1395/1801948451.py:14: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.multi_cell(0,10,txt="prompt prompt",ln=True)
/var/folders/lx/hzf7llr53d11btj66zf4sjnw0000gn/T/ipykernel_1395/1801948451.py:18: DeprecationWarni

In [ ]:
#export DF to tableou 
df_selected_income=df_selected_income.drop(columns=["Posting Date"])
df_selected_spedings=df_selected_spedings.drop(columns=["Posting Date"])

df_selected_income["Data of Transaction"]=pd.to_datetime(df_selected_income["Data of Transaction"])
df_selected_spedings["Data of Transaction"]=pd.to_datetime(df_selected_spedings["Data of Transaction"])



df_selected_income.to_csv("Data_for_tableau/Income.csv",index=False)
df_SubCat_income_to_analysis.to_csv("Data_for_tableau/Income_Sub_Cat_grupped.csv",index=False)
df_selected_spedings.to_csv("Data_for_tableau/Spendings.csv",index=False)
df_Cat_spendings_to_analysis.to_csv("Data_for_tableau/Spendings_Cat_groupped.csv",index=False)
df_SubCat_spendings_to_analysis.to_csv("Data_for_tableau/Spendings_SubCat_groupped.csv",index=False)



